### Import packages

In [1]:
import numpy as np
import anndata as ad
import pandas as pd
import scanpy as sc
import scipy
import os
import seaborn as sns
import matplotlib as mpl
from matplotlib import rcParams
import matplotlib.pyplot as plt
import gc
from scipy.stats import pearsonr
import os
import scanpy as sc
import muon as mu
from scipy.sparse import issparse, csr_matrix
from typing import Optional
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm

from matplotlib.colors import ListedColormap
from matplotlib.ticker import MaxNLocator
from scipy.cluster.hierarchy import linkage, leaves_list
from scipy.sparse import issparse
from scipy.spatial import distance_matrix
import warnings

warnings.filterwarnings("ignore")

gc.enable()

/home/in68/miniconda3/envs/cell2loc_env/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/in68/miniconda3/envs/cell2loc_env/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/in68/miniconda3/envs/cell2loc_env/lib/python3.9/site-pac

In [2]:
from utility_scripts import *

## Databases: 

In [6]:
#Load and process the gene annotation file to use metadata for analysis e.g. Biotype % calculations
# Read the .tsv file
gtf_info = pd.read_csv("Annotation_files/GRCm39_GENCODEM32_ReoT1L_gene_info_gtf.tsv", sep="\t")
gtf_info = gtf_info.drop_duplicates(subset=['GENEID'], keep='first')

# Remove period/decimal suffixes from ensembl IDs
gtf_info['GeneID'] = gtf_info['GENEID'].str.split('.').str[0]

# Extract chromosome value before the colon
gtf_info['chr'] = gtf_info['Chromosome'].str.split(':').str[0]

# Add '.1' to the Reovirus gene names to match the gtf file...
gtf_info["GENEID"] = [f"{GENE}.1" if 'T1LReovirus' in GENE else GENE for GENE in gtf_info['GENEID']]

print(f"Using {len(gtf_info['GENEID'].unique())} ensembl IDs, from {gtf_info['GeneSymbol'].nunique()} genes, and {gtf_info['Biotype'].nunique()} biotypes...\n")

# Display the first few rows
print(gtf_info.head())



#Add NCBI taxonomic ID file as dataframe to build taxonomic-level specific adata objects 
taxid = pd.read_csv('Annotation_files/ncbi_lineages_2024-01-19.csv')
Taxonomic_df = taxid.copy()
def add_prefix(x):
    return f'taxid_{x}'

# Applying function to 'tax_id' column
Taxonomic_df['tax_id'] = Taxonomic_df['tax_id'].apply(add_prefix)

# Display the updated DataFrame
print(Taxonomic_df.head())

Using 56973 ensembl IDs, from 56825 genes, and 38 biotypes...

                 GENEID     GeneSymbol            Chromosome  \
0  ENSMUSG00000102693.2  4933401J01Rik  chr1:3143476-3144545   
1  ENSMUSG00000064842.3        Gm26206  chr1:3172239-3172348   
2  ENSMUSG00000051951.6           Xkr4  chr1:3276124-3741721   
3  ENSMUSG00000102851.2        Gm18956  chr1:3322980-3323459   
4  ENSMUSG00000103377.2        Gm37180  chr1:3435954-3438772   

                Biotype Strand              GeneID   chr  
0                   TEC      +  ENSMUSG00000102693  chr1  
1                 snRNA      +  ENSMUSG00000064842  chr1  
2        protein_coding      -  ENSMUSG00000051951  chr1  
3  processed_pseudogene      +  ENSMUSG00000102851  chr1  
4                   TEC      -  ENSMUSG00000103377  chr1  
    tax_id superkingdom          phylum                class  \
0  taxid_1          NaN             NaN                  NaN   
1  taxid_2     Bacteria             NaN                  NaN   
2  tax

### Create objects 

In [8]:
taxid_to_delete = ['taxid_1', 'taxid_10090', 'taxid_119060', 'taxid_1211813', 'taxid_1224',
       'taxid_1236', 'taxid_1239', 'taxid_1268', 'taxid_1269', 'taxid_1279',
       'taxid_1280', 'taxid_1301', 'taxid_131567', 'taxid_1351', 'taxid_1385',
       'taxid_1396', 'taxid_1491', 'taxid_153135', 'taxid_1624',
       'taxid_165779', 'taxid_1716', 'taxid_1745854', 'taxid_1747',
       'taxid_1760', 'taxid_1783272', 'taxid_1789224', 'taxid_1853699',
       'taxid_186817', 'taxid_1883', 'taxid_190721', 'taxid_1912216',
       'taxid_191302', 'taxid_1916956', 'taxid_2', 'taxid_201174',
       'taxid_2049881', 'taxid_2184519', 'taxid_2306583', 'taxid_2618356',
       'taxid_2654547', 'taxid_2712222', 'taxid_2722752', 'taxid_2726118',
       'taxid_2782662', 'taxid_28211', 'taxid_28216', 'taxid_286',
       'taxid_2862880', 'taxid_2892442', 'taxid_2986923', 'taxid_2986930',
       'taxid_314146', 'taxid_31957', 'taxid_31989', 'taxid_32207',
       'taxid_33011', 'taxid_33958', 'taxid_358', 'taxid_36845', 'taxid_40324',
       'taxid_42906', 'taxid_43675', 'taxid_44283', 'taxid_446469',
       'taxid_465541', 'taxid_469', 'taxid_471', 'taxid_48736', 'taxid_524884',
       'taxid_525919', 'taxid_543', 'taxid_55601', 'taxid_564137', 'taxid_570',
       'taxid_573', 'taxid_59732', 'taxid_80840', 'taxid_82541', 'taxid_85006',
       'taxid_881', 'taxid_91061', 'taxid_91347', 'taxid_93386', 'taxid_9606',
       'taxid_976']

In [9]:
# Load the taxonomic identifiers for bacteria from a DataFrame
bacteria_taxids = Taxonomic_df[Taxonomic_df['superkingdom'] == 'Bacteria']['tax_id']
viruses_taxids = Taxonomic_df[Taxonomic_df['superkingdom'] == 'Viruses']['tax_id']
archaea_taxids = Taxonomic_df[Taxonomic_df['superkingdom'] == 'Archaea']['tax_id']

# Define the base directory where the host and microbial data are stored
base_dir = "/workdir/in68/microSTRS/results_out_Mar24/Mar24_KrakenDB/"
spatial_coord_path = "/workdir/lt425/202403_visium/visium-v1_coordinates.txt"
# List all the sample identifiers
samples = ['STRS_A',
           'STRS_B', 'STRS_C', 'STRS_D','Visium_A', 'Visium_B', 'Visium_C', 'Visium_D'
          ]
# Dictionary to store the combined data objects (MuData) for each sample
mudata_dict = {}

# Process data for each sample
for sample in samples:
    try:
        # Assemble the directory path for the host data for the current sample
        host_data_dir = os.path.join(base_dir, f"Sample_{sample}_solo/Solo.out/GeneFull/raw/")
        spliced_data_dir = os.path.join(base_dir, f"Sample_{sample}_solo/Solo.out/Velocyto/raw/spliced.mtx.gz")
        unspliced_data_dir = os.path.join(base_dir, f"Sample_{sample}_solo/Solo.out/Velocyto/raw/unspliced.mtx.gz")
        
        # Check if the host data directory exists; skip the sample if it doesn't
        if not os.path.exists(host_data_dir):
            print(f"Host data directory does not exist: {host_data_dir}")
            continue

        # Read the host gene expression data using the 10x Genomics format
        adata_host = sc.read_10x_mtx(host_data_dir, var_names='gene_symbols')
        adata_host = normalize_spatial_coordinates(adata_host, spatial_coord_path)
        adata_host.raw = adata_host.copy()
        # Add biotypes percentage and total counts to the host data
        add_biotypes_pct(adata_host, gtf_info)
        adata_host.obs['Total_counts'] = adata_host.X.sum(axis=1)
        
        # Normalize total gene expression and log transform for host data
        sc.pp.normalize_total(adata_host, target_sum=1e4)
        sc.pp.log1p(adata_host)

        # Read spliced and unspliced data
        adata_spliced = sc.read_mtx(spliced_data_dir)
        adata_unspliced = sc.read_mtx(unspliced_data_dir)
        adata_spliced = adata_spliced.T
        adata_unspliced = adata_unspliced.T
        adata_spliced.obs = adata_host.obs.copy()
        adata_unspliced.obs = adata_host.obs.copy()
        adata_spliced.var = adata_host.var.copy()
        adata_unspliced.var = adata_host.var.copy()
        adata_host.obs['Total_counts_spliced'] = adata_spliced.X.sum(axis=1)
        adata_host.obs['Total_counts_unspliced'] = adata_unspliced.X.sum(axis=1)
        adata_host.obs['Unspliced.pct'] = adata_host.obs['Total_counts_unspliced']/(adata_host.obs['Total_counts_spliced']+adata_host.obs['Total_counts_unspliced'])

        # Construct the path for microbial data and verify it
        microbial_data_path = os.path.join(base_dir, f"Sample_{sample}_solo/{sample}_microbe.h5ad")
        
        # Check if the microbial data file exists; skip the sample if it doesn't
        if not os.path.exists(microbial_data_path):
            print(f"Microbial data file does not exist: {microbial_data_path}")
            continue

        # Read the microbial data
        adata_unmapped = sc.read_h5ad(microbial_data_path)
        adata_unmapped = normalize_spatial_coordinates(adata_unmapped, spatial_coord_path)

        # Store the raw microbial data
        adata_unmapped.raw = adata_unmapped.copy()
        taxonomy_tmp = Taxonomic_df.copy()
        adata_unmapped.obs['Total_counts_unfiltered'] = adata_unmapped.X.sum(axis=1)

        # Filter the microbial data to include only bacteria
        mask = adata_unmapped.var.index.isin(bacteria_taxids)
        adata_bacteria = adata_unmapped[:, mask].copy()
        adata_bacteria.obs['Total_counts_unfiltered'] = adata_bacteria.X.sum(axis=1)

        # Delete the taxids that are found in controls
        mask2 = adata_bacteria.var.index.isin(taxid_to_delete)
        adata_bacteria = adata_bacteria[:, ~mask2].copy()


        
        # Filter the microbial data to include only virus
        mask = adata_unmapped.var.index.isin(viruses_taxids)
        adata_virus = adata_unmapped[:, mask].copy()
        adata_virus.obs['Total_counts_unfiltered'] = adata_virus.X.sum(axis=1)

        # Delete the taxids that are found in controls from virus
        mask2 = adata_virus.var.index.isin(taxid_to_delete)
        adata_virus = adata_virus[:, ~mask2].copy()
        adata_virus.obs['Total_counts'] = adata_virus.X.sum(axis=1)


        
        # Filter the microbial data to include only ARCHAEA
        mask = adata_unmapped.var.index.isin(archaea_taxids)
        adata_archaea = adata_unmapped[:, mask].copy()
        adata_archaea.obs['Total_counts_unfiltered'] = adata_archaea.X.sum(axis=1)

        # Delete the taxids that are found in controls from virus
        mask2 = adata_archaea.var.index.isin(taxid_to_delete)
        adata_archaea = adata_archaea[:, ~mask2].copy()
        adata_archaea.obs['Total_counts'] = adata_archaea.X.sum(axis=1)


        
        adata_host.obs['Total_molecules_detected'] = adata_host.obs['Total_counts'] + adata_unmapped.obs['Total_counts_unfiltered']
        adatas = {'host': adata_host, 'bacteria': adata_bacteria.copy(),
                  'viruses': adata_virus.copy(),'archaea': adata_archaea.copy(), 
                  'unmapped_all': adata_unmapped.copy(), 'spliced': adata_spliced, 'unspliced': adata_unspliced}



        
        # Process each taxonomic level bacteria
        for tax_level in ['species', 'genus', 'family', 'phylum']:
            # adata_bacteria.uns['taxonomy'][tax_level] = adata_ba el].fillna('unclassified')
            tax_id_to_name = dict(zip(Taxonomic_df['tax_id'], Taxonomic_df[tax_level]))
            adata_bacteria.var[tax_level] = adata_bacteria.var.index.map(tax_id_to_name).fillna('unclassified')
            adata_df = pd.DataFrame(adata_bacteria.X.toarray(), columns=adata_bacteria.var[tax_level])
            aggregated_data = adata_df.groupby(adata_df.columns, axis=1).sum()

            new_vars = pd.DataFrame(index=aggregated_data.columns)
            X_mat = csr_matrix(np.round(aggregated_data.values).astype(np.float32))

            new_adata_bacteria = sc.AnnData(X=X_mat, var=new_vars, obs=adata_bacteria.obs)

            # Add the raw component
            new_adata_bacteria.raw = new_adata_bacteria.copy()

            new_adata_bacteria.obs['Total_counts'] = new_adata_bacteria.X.sum(axis=1)
            unclassified_mask = new_adata_bacteria.var.index == 'unclassified'
            new_adata_bacteria.obs['Total_counts_unclassified'] = new_adata_bacteria[:, unclassified_mask].X.sum(axis=1)
            new_adata_bacteria.obs['Total_counts_classified'] = new_adata_bacteria.obs['Total_counts'] - new_adata_bacteria.obs['Total_counts_unclassified']
            # Filtering step after calculating Total_counts
            # total_counts = new_adata_bacteria.obs['Total_counts'].sum()
            # taxon_percentages = new_adata_bacteria.X.sum(axis=0) / total_counts * 100

            # # Identify taxa that are <= 0.01% and remove them
            # low_abundance_mask = taxon_percentages <= 0.01
            # new_adata_bacteria = new_adata_bacteria[:, ~low_abundance_mask].copy()
            # # Remove 'unclassified' variable if it exists
            # if 'unclassified' in new_adata_bacteria.var_names:
            #     unclassified_mask = new_adata_bacteria.var_names == 'unclassified'
            #     new_adata_bacteria = new_adata_bacteria[:, ~unclassified_mask].copy()


            
            # Normalize counts and log transform
            total_microbial_counts = new_adata_bacteria.X.sum()
            new_adata_bacteria.X = new_adata_bacteria.X / total_microbial_counts * 10000
            new_adata_bacteria = normalize_spatial_coordinates(new_adata_bacteria, spatial_coord_path)
            spot_counts = new_adata_bacteria.X > 0
            unique_genes_per_spot = spot_counts.sum(axis=1)
            new_adata_bacteria.obs['unique_taxa_per_spot'] = unique_genes_per_spot
            sc.pp.log1p(new_adata_bacteria)

            adatas[tax_level] = new_adata_bacteria
            # Delete intermediate variables to free up memory
            del adata_df, aggregated_data, new_vars, X_mat, new_adata_bacteria, tax_id_to_name
       

        # Process each taxonomic level for virus
        for tax_level in ['species', 'genus', 'family', 'phylum']:
            # adata_virus.uns['taxonomy'][tax_level] = adata_virus.uns['taxonomy'][tax_level].fillna('unclassified')
            tax_id_to_name = dict(zip(Taxonomic_df['tax_id'], Taxonomic_df[tax_level]))
            adata_virus.var[tax_level] = adata_virus.var.index.map(tax_id_to_name).fillna('unclassified')
            adata_df = pd.DataFrame(adata_virus.X.toarray(), columns=adata_virus.var[tax_level])
            aggregated_data = adata_df.groupby(adata_df.columns, axis=1).sum()

            new_vars = pd.DataFrame(index=aggregated_data.columns)
            X_mat = csr_matrix(np.round(aggregated_data.values).astype(np.float32))

            new_adata_virus = sc.AnnData(X=X_mat, var=new_vars, obs=adata_virus.obs)

            # Add the raw component
            new_adata_virus.raw = new_adata_virus.copy()

            new_adata_virus.obs['Total_counts'] = new_adata_virus.X.sum(axis=1)
            unclassified_mask = new_adata_virus.var.index == 'unclassified'
            new_adata_virus.obs['Total_counts_unclassified'] = new_adata_virus[:, unclassified_mask].X.sum(axis=1)
            new_adata_virus.obs['Total_counts_classified'] = new_adata_virus.obs['Total_counts'] - new_adata_virus.obs['Total_counts_unclassified']

            # Normalize counts and log transform
            total_microbial_counts = new_adata_virus.X.sum()
            new_adata_virus.X = new_adata_virus.X / total_microbial_counts * 10000
            new_adata_virus = normalize_spatial_coordinates(new_adata_virus, spatial_coord_path)
            spot_counts = new_adata_virus.X > 0
            unique_genes_per_spot = spot_counts.sum(axis=1)
            new_adata_virus.obs['unique_taxa_per_spot'] = unique_genes_per_spot
            sc.pp.log1p(new_adata_virus)
            tax_level_to_use = f"{tax_level}_virus"
            adatas[tax_level_to_use] = new_adata_virus
            # Delete intermediate variables to free up memory
            del adata_df, aggregated_data, new_vars, X_mat, new_adata_virus, tax_id_to_name
            
        # Process each taxonomic level for archaea
        for tax_level in ['species', 'genus', 'family', 'phylum']:
            # adata_archaea.uns['taxonomy'][tax_level] = adata_archaea.uns['taxonomy'][tax_level].fillna('unclassified')
            tax_id_to_name = dict(zip(Taxonomic_df['tax_id'], Taxonomic_df[tax_level]))
            adata_archaea.var[tax_level] = adata_archaea.var.index.map(tax_id_to_name).fillna('unclassified')
            adata_df = pd.DataFrame(adata_archaea.X.toarray(), columns=adata_archaea.var[tax_level])
            aggregated_data = adata_df.groupby(adata_df.columns, axis=1).sum()

            new_vars = pd.DataFrame(index=aggregated_data.columns)
            X_mat = csr_matrix(np.round(aggregated_data.values).astype(np.float32))

            new_adata_archaea = sc.AnnData(X=X_mat, var=new_vars, obs=adata_archaea.obs)

            # Add the raw component
            new_adata_archaea.raw = new_adata_archaea.copy()

            new_adata_archaea.obs['Total_counts'] = new_adata_archaea.X.sum(axis=1)
            unclassified_mask = new_adata_archaea.var.index == 'unclassified'
            new_adata_archaea.obs['Total_counts_unclassified'] = new_adata_archaea[:, unclassified_mask].X.sum(axis=1)
            new_adata_archaea.obs['Total_counts_classified'] = new_adata_archaea.obs['Total_counts'] - new_adata_archaea.obs['Total_counts_unclassified']

            # Normalize counts and log transform
            total_microbial_counts = new_adata_archaea.X.sum()
            new_adata_archaea.X = new_adata_archaea.X / total_microbial_counts * 10000
            new_adata_archaea = normalize_spatial_coordinates(new_adata_archaea, spatial_coord_path)
            spot_counts = new_adata_archaea.X > 0
            unique_genes_per_spot = spot_counts.sum(axis=1)
            new_adata_archaea.obs['unique_taxa_per_spot'] = unique_genes_per_spot
            sc.pp.log1p(new_adata_archaea)
            tax_level_to_use = f"{tax_level}_archaea"
            adatas[tax_level_to_use] = new_adata_archaea



            
            # Delete intermediate variables to free up memory
            del adata_df, aggregated_data, new_vars, X_mat, new_adata_archaea, tax_id_to_name

        # Store the combined data object for each sample
        mudata_dict[sample] = mu.MuData(adatas)

        # Delete intermediate variables to free up memory
        del adata_host, adata_unmapped, adata_bacteria, adatas, mask, mask2, adata_spliced, adata_unspliced

    except Exception as e:
        print(f"An error occurred while processing sample {sample}: {e}")

# Output which samples have been successfully processed
print("Processed samples:", list(mudata_dict.keys()))
gc.collect()


--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
Added spatial coordinates for 4992 cells.
No gene_version1 genes found...
normalizing counts per cell
    finished (0:00:00)
Added spatial coordinates for 4991 cells.
Added spatial coordinates for 4991 cells.
Added spatial coordinates for 4991 cells.
Added spatial coordinates for 4991 cells.
Added spatial coordinates for 4991 cells.
Added spatial coordinates for 4991 cells.
Added spatial coordinates for 4991 cells.
Added spatial coordinates for 4991 cells.
Added spatial coordinates for 4991 cells.
Added spatial coordinates for 4991 cells.
Added spatial coordinates for 4991 cells.
Added spatial coordinates for 4991 cells.
Added spatial coordinates for 4991 cells.
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
Added spatial coordinates for 4992 cells.
No gene_version1 genes found...
normalizing counts per cell
 

10075

In [ ]:
# Define paths to JSON files and barcode file for each sample
samples_json_files = {
    'STRS_A': [
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/STRS_2024/V13A05-372-A1_Outside.json",
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/STRS_2024/V13A05-372-A1_Lumen.json",
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/STRS_2024/V13A05-372-A1_Tissue.json",
    ],
    'STRS_B': [
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/STRS_2024/V13A05-372-B1_Outside.json",
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/STRS_2024/V13A05-372-B1_Lumen.json",
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/STRS_2024/V13A05-372-B1_Tissue.json",
    ],
    'STRS_C': [
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/STRS_2024/V13A05-372-C1_Outside.json",
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/STRS_2024/V13A05-372-C1_Lumen.json",
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/STRS_2024/V13A05-372-C1_Tissue.json",
    ],
    'STRS_D': [
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/STRS_2024/V13A05-372-D1_Outside.json",
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/STRS_2024/V13A05-372-D1_Lumen.json",
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/STRS_2024/V13A05-372-D1_Tissue.json",
    ],    
    'Visium_A': [
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/Visium_2024/V13A05-407-A1_Outside.json",
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/Visium_2024/V13A05-407-A1_Lumen.json",
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/Visium_2024/V13A05-407-A1_Tissue.json",
    ],
    'Visium_B': [
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/Visium_2024/V13A05-407-B1_Outside.json",
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/Visium_2024/V13A05-407-B1_Lumen.json",
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/Visium_2024/V13A05-407-B1_Tissue.json",
    ],
    'Visium_C': [
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/Visium_2024/V13A05-407-C1_Outside.json",
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/Visium_2024/V13A05-407-C1_Lumen.json",
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/Visium_2024/V13A05-407-C1_Tissue.json",
    ],
    'Visium_D': [
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/Visium_2024/V13A05-407-D1_Outside.json",
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/Visium_2024/V13A05-407-D1_Lumen.json",
        "/workdir/in68/micro_STRS_github/microSTRS/Analysis/Low_resolution/tissue_location_files/Visium_2024/V13A05-407-D1_Tissue.json",
    ]    
}

# Path to the barcode file
barcode_file = spatial_coord_path

# Example MuData objects (assuming these are pre-loaded)


for sample, json_files in samples_json_files.items():
    # Process the JSON files and create the DataFrame
    barcode_presence_df = add_histology_info_visium(json_files, barcode_file)
    
    # Add the classification DataFrame to the `obsm` slot of each modality in the MuData object
    mudata = mudata_dict[sample]
    for modality in mudata.mod.keys():
        adata = mudata[modality]
        adata.obsm['classification'] = barcode_presence_df.loc[adata.obs_names].fillna(False)

# Now each MuData object in `mudata_objects` has the barcode classification added to its modalities' `obsm` slots.


In [11]:
# Also we are gonna add the cell2loc as cluster 
adata = sc.read_h5ad("/fs/cbsuvlaminck3/workdir/in68/STRS-HD/Gut_STRS_060524/CoLocatedComb/CoLocatedGroupsSklearnNMF_11642locations_21factors/anndata/sp.h5ad")
adata.obs[adata.uns['mod']['factor_names']] = adata.obsm['q05_cell_abundance_w_sf']
adata.obsm['q05_cell_proportions'] = adata.obsm['q05_cell_abundance_w_sf'].div(adata.obsm['q05_cell_abundance_w_sf'].sum(axis=1), axis=0)

# add 5% quantile, representing confident cell abundance, 'at least this amount is present',
# to adata.obs with nice names for plotting

adata.obs["total_abundance"] = adata.obsm['q05_cell_abundance_w_sf'].sum(axis = 1)
# adata_obj.obs[adata_obj.uns['mod']['factor_names']] = adata_obj.obsm['q05_cell_abundance_w_sf']
adata.obs[adata.uns['mod']['factor_names']] = adata.obsm['q05_cell_proportions']
adata.obs["max_pred"] = adata.obs[adata.uns['mod']['factor_names']].max(axis=1)
adata.obs["max_pred_celltype"] = adata.obs[adata.uns['mod']['factor_names']].idxmax(axis=1)
adata.obs["max_pred_celltype"] = adata.obs["max_pred_celltype"].astype('category')

# Function to revert observation names to original
def revert_obs_names(obs_names):
    return [name.split('_')[0] for name in obs_names]

# Dictionary to store the 'cell2loc' modality for each sample
cell2loc_dict = {}

# Loop through each sample in the mudata_dict
for sample in mudata_dict.keys():
    # Extract the data for the current sample from the combined AnnData
    sample_data = adata[adata.obs['Sample'] == sample].copy()

    # Revert observation names to original for both obs and obsm
    reverted_obs_names = revert_obs_names(sample_data.obs.index.tolist())
    sample_data.obs.index = reverted_obs_names
    sample_data.obsm['q05_cell_abundance_w_sf'].index = reverted_obs_names
    sample_data.obsm['spatial'] = pd.DataFrame(sample_data.obsm['spatial'], index=reverted_obs_names)
    sample_data.obsm['classification'] = pd.DataFrame(sample_data.obsm['classification'], index=reverted_obs_names)

    # Prepare the new AnnData for the 'cell2loc' modality
    cell2loc_adata = sc.AnnData(
        X=sample_data.obsm['q05_cell_abundance_w_sf'].values,
        obs=sample_data.obs,
        var=pd.DataFrame(index=sample_data.obsm['q05_cell_abundance_w_sf'].columns)
    )

    # Ensure the indices are the same order for spatial and classification
    spatial_data = sample_data.obsm['spatial'].values
    classification_data = sample_data.obsm['classification'].values

    # Assign obsm values
    cell2loc_adata.obsm['spatial'] = spatial_data
    cell2loc_adata.obsm['classification'] = classification_data

    # Store the new AnnData object in the dictionary
    cell2loc_dict[sample] = cell2loc_adata

# Output which samples have been successfully processed
for sample, adata in cell2loc_dict.items():
    print(f"Sample {sample}:")
    print(adata)

import muon as mu

# Loop through each sample in the mudata_dict
for sample in mudata_dict.keys():
    # Check if the sample exists in the cell2loc_dict
    if sample in cell2loc_dict:
        # Get the existing MuData object
        mudata = mudata_dict[sample]
        
        # Remove the existing 'cell2loc' modality if it exists
        if 'cell2loc' in mudata.mod:
            del mudata.mod['cell2loc']
        
        # Add the new 'cell2loc' modality
        mudata.mod['cell2loc'] = cell2loc_dict[sample]
        
        # Replace the existing MuData object with the updated one
        mudata_dict[sample] = mudata

# Output the updated MuData objects to confirm the addition
for sample, mudata in mudata_dict.items():
    print(f"Sample {sample}:")
    print(mudata)
# Loop through each sample in the mudata_dict to add the 'spatial' obsm to 'cell2loc' modality
for sample in mudata_dict.keys():
    # Get the existing MuData object
    mudata = mudata_dict[sample]
    
    # Ensure 'cell2loc' modality exists in the MuData object
    if 'cell2loc' in mudata.mod:
        # Get the 'host' and 'cell2loc' AnnData objects
        host_adata = mudata['host']
        cell2loc_adata = mudata['cell2loc']
        
        # Ensure the index names match before copying 'spatial' obsm
        if set(cell2loc_adata.obs.index).issubset(set(host_adata.obs.index)):
            cell2loc_adata.obsm['spatial'] = host_adata.obsm['spatial'][host_adata.obs.index.isin(cell2loc_adata.obs.index), :]
        else:
            print(f"Warning: Index mismatch for sample {sample}")

        # Replace the updated 'cell2loc' AnnData object in the MuData object
        mudata.mod['cell2loc'] = cell2loc_adata
        
        # Replace the existing MuData object with the updated one
        mudata_dict[sample] = mudata

# Output the updated MuData objects to confirm the addition
for sample, mudata in mudata_dict.items():
    print(f"Sample {sample}:")
    print(mudata)

# Loop through each sample in the mudata_dict
for sample in mudata_dict.keys():
    # Extract the host and cell2loc data
    host_data = mudata_dict[sample]['host']
    cell2loc_data = mudata_dict[sample]['cell2loc']
    
    # Filter the classification data based on the indices in the cell2loc obs
    filtered_classification = host_data.obsm['classification'].loc[cell2loc_data.obs.index]
    
    # Assign the filtered classification data to the cell2loc modality
    cell2loc_data.obsm['classification'] = filtered_classification

    # Optionally, verify the operation
    print(f"Sample {sample}:")
    print(cell2loc_data.obsm['classification'].head())

# Output the updated MuData objects to confirm the addition
for sample, mudata in mudata_dict.items():
    print(f"Sample {sample}:")
    print(mudata['cell2loc'])

# Original dictionary with colors
celltype_color_dict = {
    'q05cell_abundance_w_sf_B cells': '#183E9F',
    'q05cell_abundance_w_sf_CD8 T cells': '#F978E9',
    'q05cell_abundance_w_sf_Cholangiocytes': '#747475',
    'q05cell_abundance_w_sf_Enteroendocrine cells': '#F2E64C',
    'q05cell_abundance_w_sf_Epithelial cells': '#FC8900',
    'q05cell_abundance_w_sf_Erythroid-like and erythroid precursor cells': '#5FCCC0',
    'q05cell_abundance_w_sf_Glandular epithelial cells': '#B8F4F2',
    'q05cell_abundance_w_sf_Goblet cells': '#8FF469',
    'q05cell_abundance_w_sf_Immature Enterocytes 1': '#B6BBE0',
    'q05cell_abundance_w_sf_Macrophages': '#BCAE04',
    'q05cell_abundance_w_sf_Mature Enterocytes 1': '#C95F7B',
    'q05cell_abundance_w_sf_Mature Enterocytes 2': '#C95F7B',
    'q05cell_abundance_w_sf_Mature Enterocytes 3': '#C95F7B',
    'q05cell_abundance_w_sf_Mature Enterocytes 4': '#C95F7B',
    'q05cell_abundance_w_sf_Mature Enterocytes 5': '#C95F7B',
    'q05cell_abundance_w_sf_Memory T cells': '#6ABF4B',
    'q05cell_abundance_w_sf_Mesothelial cells': '#E8BB93',
    'q05cell_abundance_w_sf_Paneth cells': '#6B845E',
    'q05cell_abundance_w_sf_TA cells': '#CC0202',
    'q05cell_abundance_w_sf_Tuft cells': '#7D199E',
    'q05cell_abundance_w_sf_pDCs': '#D9E9E7',
    'Others': '#747475'
}

# Remove prefix
prefix = 'q05cell_abundance_w_sf_'
celltype_color_dict_no_prefix = {k.replace(prefix, ''): v for k, v in celltype_color_dict.items()}

# Function to assign colors to unique cell types in a given AnnData object
def assign_colors(adata, celltype_color_dict):
    unique_celltypes = np.unique(adata.obs['max_pred_celltype'])
    celltype_colors = [celltype_color_dict.get(celltype, celltype_color_dict['Others']) for celltype in unique_celltypes]
    adata.uns['max_pred_celltype_colors'] = celltype_colors
    return unique_celltypes, celltype_colors

# Loop through each sample in the mudata_dict and assign colors
for sample in mudata_dict.keys():
    adata = mudata_dict[sample]['cell2loc']
    unique_celltypes, celltype_colors = assign_colors(adata, celltype_color_dict_no_prefix)
    
    # # Print the assigned colors for verification
    # print(f"Sample {sample}:")
    # print("Unique cell types:", unique_celltypes)
    # print("Assigned colors:", celltype_colors)
    
    # # Plot to verify
    # plt.figure(figsize=(10, 2))
    # for i, (celltype, color) in enumerate(zip(unique_celltypes, celltype_colors)):
    #     plt.plot([], [], marker='o', markersize=10, label=celltype, color=color)
    # plt.legend(loc='center', ncol=4)
    # plt.axis('off')
    # plt.show()


Sample STRS_A:
AnnData object with n_obs × n_vars = 2701 × 21
    obs: 'pct.TEC', 'pct.snRNA', 'pct.protein_coding', 'pct.processed_pseudogene', 'pct.lncRNA', 'pct.miRNA', 'pct.snoRNA', 'pct.misc_RNA', 'pct.transcribed_unprocessed_pseudogene', 'pct.unprocessed_pseudogene', 'pct.rRNA', 'pct.transcribed_processed_pseudogene', 'pct.ribozyme', 'pct.unitary_pseudogene', 'pct.scaRNA', 'pct.pseudogene', 'pct.transcribed_unitary_pseudogene', 'pct.translated_unprocessed_pseudogene', 'pct.TR_V_gene', 'pct.TR_V_pseudogene', 'pct.TR_D_gene', 'pct.TR_J_gene', 'pct.TR_C_gene', 'pct.TR_J_pseudogene', 'pct.IG_LV_gene', 'pct.IG_V_gene', 'pct.IG_V_pseudogene', 'pct.IG_J_gene', 'pct.IG_C_gene', 'pct.sRNA', 'pct.scRNA', 'pct.IG_C_pseudogene', 'pct.IG_D_gene', 'pct.IG_D_pseudogene', 'pct.IG_pseudogene', 'pct.Mt_tRNA', 'pct.Mt_rRNA', 'Total_counts', 'Total_counts_spliced', 'Total_counts_unspliced', 'Unspliced_Spliced_ratio', 'Ratio', 'Total_molecules_detected', 'Sample', 'assay', 'n_counts', '_indices', '_s

In [18]:
## Add the heart controls w and wo polyA
# Load the taxonomic identifiers for bacteria from a DataFrame
bacteria_taxids = Taxonomic_df[Taxonomic_df['superkingdom'] == 'Bacteria']['tax_id']

# Define the base directory where the host and microbial data are stored
base_dir = "/fs/cbsuvlaminck2/workdir/in68/microSTRS/results_out_Downloaded/Controls/"

# List all the sample identifiers
samples = ['SRR17048630', 'Vis_yPAP_3A_S1_L001']

# Dictionary to store the combined data objects (MuData) for each sample
mudata_dict_3 = {}

for sample in samples:
    try:
        # Assemble the directory path for the host data for the current sample
        host_data_dir = os.path.join(base_dir, f"{sample}_solo/Solo.out/GeneFull/raw/")
        spliced_data_dir = os.path.join(base_dir, f"{sample}_solo/Solo.out/Velocyto/raw/spliced.mtx.gz")
        unspliced_data_dir = os.path.join(base_dir, f"{sample}_solo/Solo.out/Velocyto/raw/unspliced.mtx.gz")
        
        # Check if the host data directory exists; skip the sample if it doesn't
        if not os.path.exists(host_data_dir):
            print(f"Host data directory does not exist: {host_data_dir}")
            continue

        # Read the host gene expression data using the 10x Genomics format
        adata_host = sc.read_10x_mtx(host_data_dir, var_names='gene_symbols')
        adata_host = normalize_spatial_coordinates(adata_host, spatial_coord_path)
        adata_host.raw = adata_host.copy()
        
        # Add biotypes percentage and total counts to the host data
        add_biotypes_pct(adata_host, gtf_info)
        adata_host.obs['Total_counts'] = adata_host.X.sum(axis=1)

        # Normalize total gene expression and log transform for host data
        sc.pp.normalize_total(adata_host, target_sum=1e4)
        sc.pp.log1p(adata_host)

        # Read spliced and unspliced data
        adata_spliced = sc.read_mtx(spliced_data_dir)
        adata_unspliced = sc.read_mtx(unspliced_data_dir)
        adata_spliced = adata_spliced.T
        adata_unspliced = adata_unspliced.T
        adata_spliced.obs = adata_host.obs.copy()
        adata_unspliced.obs = adata_host.obs.copy()
        adata_spliced.var = adata_host.var.copy()
        adata_unspliced.var = adata_host.var.copy()
        adata_host.obs['Total_counts_spliced'] = adata_spliced.X.sum(axis=1)
        adata_host.obs['Total_counts_unspliced'] = adata_unspliced.X.sum(axis=1)
        adata_host.obs['Unspliced.pct'] = adata_host.obs['Total_counts_unspliced']/(adata_host.obs['Total_counts_spliced']+adata_host.obs['Total_counts_unspliced'])

        # Construct the path for microbial data and verify it
        microbial_data_path = os.path.join(base_dir, f"{sample}_solo/microbe.h5ad")
        
        # Check if the microbial data file exists; skip the sample if it doesn't
        if not os.path.exists(microbial_data_path):
            print(f"Microbial data file does not exist: {microbial_data_path}")
            continue

        # Read the microbial data
        adata_unmapped = sc.read_h5ad(microbial_data_path)
        adata_unmapped = normalize_spatial_coordinates(adata_unmapped, spatial_coord_path)

        # Store the raw microbial data
        adata_unmapped.raw = adata_unmapped.copy()
        # adata_unmapped.uns['taxonomy'] = df.copy()
        adata_unmapped.obs['Total_counts_unfiltered'] = adata_unmapped.X.sum(axis=1)

        # Filter the microbial data to include only bacteria
        mask = adata_unmapped.var.index.isin(bacteria_taxids)
        adata_bacteria = adata_unmapped[:, mask].copy()
        adata_bacteria.obs['Total_counts_unfiltered'] = adata_bacteria.X.sum(axis=1)

        # Delete the taxids that are found in controls
        mask2 = adata_bacteria.var.index.isin(taxid_to_delete)
        adata_bacteria = adata_bacteria[:, ~mask2].copy()


        
        # Filter the microbial data to include only virus
        mask = adata_unmapped.var.index.isin(viruses_taxids)
        adata_virus = adata_unmapped[:, mask].copy()
        adata_virus.obs['Total_counts_unfiltered'] = adata_virus.X.sum(axis=1)

        # Delete the taxids that are found in controls from virus
        mask2 = adata_virus.var.index.isin(taxid_to_delete)
        adata_virus = adata_virus[:, ~mask2].copy()
        adata_virus.obs['Total_counts'] = adata_virus.X.sum(axis=1)


        
        # Filter the microbial data to include only ARCHAEA
        mask = adata_unmapped.var.index.isin(archaea_taxids)
        adata_archaea = adata_unmapped[:, mask].copy()
        adata_archaea.obs['Total_counts_unfiltered'] = adata_archaea.X.sum(axis=1)

        # Delete the taxids that are found in controls from virus
        mask2 = adata_archaea.var.index.isin(taxid_to_delete)
        adata_archaea = adata_archaea[:, ~mask2].copy()
        adata_archaea.obs['Total_counts'] = adata_archaea.X.sum(axis=1)

        adata_host.obs['Total_molecules_detected'] = adata_host.obs['Total_counts'] + adata_unmapped.obs['Total_counts_unfiltered']
        adatas = {'host': adata_host, 'bacteria': adata_bacteria.copy(),
                  'viruses': adata_virus.copy(),'archaea': adata_archaea.copy(), 
                  'unmapped_all': adata_unmapped.copy(), 'spliced': adata_spliced, 'unspliced': adata_unspliced}



        
        # Process each taxonomic level bacteria
        for tax_level in ['species', 'genus', 'family', 'phylum']:
            # adata_bacteria.uns['taxonomy'][tax_level] = adata_bacteria.uns['taxonomy'][tax_level].fillna('unclassified')
            tax_id_to_name = dict(zip(Taxonomic_df['tax_id'], Taxonomic_df[tax_level]))
            adata_bacteria.var[tax_level] = adata_bacteria.var.index.map(tax_id_to_name).fillna('unclassified')
            adata_df = pd.DataFrame(adata_bacteria.X.toarray(), columns=adata_bacteria.var[tax_level])
            aggregated_data = adata_df.groupby(adata_df.columns, axis=1).sum()
            if aggregated_data.shape[1] == 0:
                print(f"No data found for bacteria at {tax_level} level. Skipping this level.")
                continue
            new_vars = pd.DataFrame(index=aggregated_data.columns)
            X_mat = csr_matrix(np.round(aggregated_data.values).astype(np.float32))

            new_adata_bacteria = sc.AnnData(X=X_mat, var=new_vars, obs=adata_bacteria.obs)

            # Add the raw component
            new_adata_bacteria.raw = new_adata_bacteria.copy()

            new_adata_bacteria.obs['Total_counts'] = new_adata_bacteria.X.sum(axis=1)
            unclassified_mask = new_adata_bacteria.var.index == 'unclassified'
            new_adata_bacteria.obs['Total_counts_unclassified'] = new_adata_bacteria[:, unclassified_mask].X.sum(axis=1)
            new_adata_bacteria.obs['Total_counts_classified'] = new_adata_bacteria.obs['Total_counts'] - new_adata_bacteria.obs['Total_counts_unclassified']

            # Normalize counts and log transform
            total_microbial_counts = new_adata_bacteria.X.sum()
            new_adata_bacteria.X = new_adata_bacteria.X / total_microbial_counts * 10000
            new_adata_bacteria = normalize_spatial_coordinates(new_adata_bacteria, spatial_coord_path)
            spot_counts = new_adata_bacteria.X > 0
            unique_genes_per_spot = spot_counts.sum(axis=1)
            new_adata_bacteria.obs['unique_taxa_per_spot'] = unique_genes_per_spot
            sc.pp.log1p(new_adata_bacteria)

            adatas[tax_level] = new_adata_bacteria
            # Delete intermediate variables to free up memory
            del adata_df, aggregated_data, new_vars, X_mat, new_adata_bacteria, tax_id_to_name            

        # Process each taxonomic level for virus
        for tax_level in ['species', 'genus', 'family', 'phylum']:
            # adata_virus.uns['taxonomy'][tax_level] = adata_virus.uns['taxonomy'][tax_level].fillna('unclassified')
            tax_id_to_name = dict(zip(Taxonomic_df['tax_id'], Taxonomic_df[tax_level]))
            adata_virus.var[tax_level] = adata_virus.var.index.map(tax_id_to_name).fillna('unclassified')
            adata_df = pd.DataFrame(adata_virus.X.toarray(), columns=adata_virus.var[tax_level])
            aggregated_data = adata_df.groupby(adata_df.columns, axis=1).sum()
            if aggregated_data.shape[1] == 0:
                print(f"No data found for bacteria at {tax_level} level. Skipping this level.")
                continue
            new_vars = pd.DataFrame(index=aggregated_data.columns)
            X_mat = csr_matrix(np.round(aggregated_data.values).astype(np.float32))

            new_adata_virus = sc.AnnData(X=X_mat, var=new_vars, obs=adata_virus.obs)

            # Add the raw component
            new_adata_virus.raw = new_adata_virus.copy()

            new_adata_virus.obs['Total_counts'] = new_adata_virus.X.sum(axis=1)
            unclassified_mask = new_adata_virus.var.index == 'unclassified'
            new_adata_virus.obs['Total_counts_unclassified'] = new_adata_virus[:, unclassified_mask].X.sum(axis=1)
            new_adata_virus.obs['Total_counts_classified'] = new_adata_virus.obs['Total_counts'] - new_adata_virus.obs['Total_counts_unclassified']

            # Normalize counts and log transform
            total_microbial_counts = new_adata_virus.X.sum()
            new_adata_virus.X = new_adata_virus.X / total_microbial_counts * 10000
            new_adata_virus = normalize_spatial_coordinates(new_adata_virus, spatial_coord_path)
            spot_counts = new_adata_virus.X > 0
            unique_genes_per_spot = spot_counts.sum(axis=1)
            new_adata_virus.obs['unique_taxa_per_spot'] = unique_genes_per_spot
            sc.pp.log1p(new_adata_virus)
            tax_level_to_use = f"{tax_level}_virus"
            adatas[tax_level_to_use] = new_adata_virus
            # Delete intermediate variables to free up memory
            del adata_df, aggregated_data, new_vars, X_mat, new_adata_virus, tax_id_to_name
            
        # Process each taxonomic level for archaea
        for tax_level in ['species', 'genus', 'family', 'phylum']:
            # adata_archaea.uns['taxonomy'][tax_level] = adata_archaea.uns['taxonomy'][tax_level].fillna('unclassified')
            tax_id_to_name = dict(zip(Taxonomic_df['tax_id'], Taxonomic_df[tax_level]))
            adata_archaea.var[tax_level] = adata_archaea.var.index.map(tax_id_to_name).fillna('unclassified')
            adata_df = pd.DataFrame(adata_archaea.X.toarray(), columns=adata_archaea.var[tax_level])
            aggregated_data = adata_df.groupby(adata_df.columns, axis=1).sum()
            if aggregated_data.shape[1] == 0:
                print(f"No data found for bacteria at {tax_level} level. Skipping this level.")
                continue
            new_vars = pd.DataFrame(index=aggregated_data.columns)
            X_mat = csr_matrix(np.round(aggregated_data.values).astype(np.float32))

            new_adata_archaea = sc.AnnData(X=X_mat, var=new_vars, obs=adata_archaea.obs)

            # Add the raw component
            new_adata_archaea.raw = new_adata_archaea.copy()

            new_adata_archaea.obs['Total_counts'] = new_adata_archaea.X.sum(axis=1)
            unclassified_mask = new_adata_archaea.var.index == 'unclassified'
            new_adata_archaea.obs['Total_counts_unclassified'] = new_adata_archaea[:, unclassified_mask].X.sum(axis=1)
            new_adata_archaea.obs['Total_counts_classified'] = new_adata_archaea.obs['Total_counts'] - new_adata_archaea.obs['Total_counts_unclassified']

            # Normalize counts and log transform
            total_microbial_counts = new_adata_archaea.X.sum()
            new_adata_archaea.X = new_adata_archaea.X / total_microbial_counts * 10000
            new_adata_archaea = normalize_spatial_coordinates(new_adata_archaea, spatial_coord_path)
            spot_counts = new_adata_archaea.X > 0
            unique_genes_per_spot = spot_counts.sum(axis=1)
            new_adata_archaea.obs['unique_taxa_per_spot'] = unique_genes_per_spot
            sc.pp.log1p(new_adata_archaea)
            tax_level_to_use = f"{tax_level}_archaea"
            adatas[tax_level_to_use] = new_adata_archaea



            
            # Delete intermediate variables to free up memory
            del adata_df, aggregated_data, new_vars, X_mat, new_adata_archaea, tax_id_to_name

        # Store the combined data object for each sample
        mudata_dict_3[sample] = mu.MuData(adatas)

        # Delete intermediate variables to free up memory
        del adata_host, adata_unmapped, adata_bacteria, adatas, mask, mask2, adata_spliced, adata_unspliced

    except Exception as e:
        print(f"An error occurred while processing sample {sample}: {e}")

# Output which samples have been successfully processed
print("Processed samples:", list(mudata_dict_3.keys()))

--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
Added spatial coordinates for 4992 cells.
No gene_version1 genes found...
normalizing counts per cell
    finished (0:00:00)
Added spatial coordinates for 4887 cells.
Added spatial coordinates for 4887 cells.
Added spatial coordinates for 4887 cells.
Added spatial coordinates for 4887 cells.
Added spatial coordinates for 4887 cells.
Added spatial coordinates for 4887 cells.
Added spatial coordinates for 4887 cells.
Added spatial coordinates for 4887 cells.
Added spatial coordinates for 4887 cells.
No data found for bacteria at species level. Skipping this level.
No data found for bacteria at genus level. Skipping this level.
No data found for bacteria at family level. Skipping this level.
No data found for bacteria at phylum level. Skipping this level.
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
Added spati

In [19]:
# Assuming mudata_dict_2 is already defined and contains your MuData objects

# Create a classification DataFrame with 'A' and 'B' as False and 'C' as True
def create_classification_df(barcodes):
    # Initialize the DataFrame
    classification_df = pd.DataFrame(False, index=barcodes, columns=['A', 'B'])
    classification_df['C'] = True
    return classification_df

# Iterate through each MuData object in mudata_dict_3
for sample, mudata in mudata_dict_3.items():
    for modality in mudata.mod.keys():
        adata = mudata[modality]
        # Create the classification DataFrame for the barcodes in adata.obs_names
        classification_df = create_classification_df(adata.obs_names)
        # Add the classification DataFrame to the `obsm` slot
        adata.obsm['classification'] = classification_df

# mudata_dict_3['SRR17048630']['host'].obsm['classification']

In [20]:
# and add them to mudata_dict
mudata_dict['Visium_heart'] = mudata_dict_3.pop('SRR17048630')

# and add them to mudata_dict
mudata_dict['STRS_heart'] = mudata_dict_3.pop('Vis_yPAP_3A_S1_L001')

In [21]:
gc.collect()

13982

## save adata objects

In [ ]:
save_dir = "/workdir/in68/microSTRS/saved_mudata"
os.makedirs(save_dir, exist_ok=True)

for sample, mdata in mudata_dict.items():
    sample_dir = os.path.join(save_dir, sample)
    os.makedirs(sample_dir, exist_ok=True)
    for mod_name, ad in mdata.mod.items():
        ad.write(os.path.join(sample_dir, f"{mod_name}.h5ad"))